<a href="https://colab.research.google.com/github/saatvik1879/nlptask/blob/main/NLPtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## preprocessing data

In [35]:
pip install nltk

In [36]:
import pandas as pd

data = pd.read_csv('paragraphs2.csv')

paragraphs = data['Paragraphs'].tolist()
# removing duplicates
paragraphs = list(set(paragraphs))

qdata = pd.read_csv('queries.csv')

queries = qdata['Queries'].tolist()

answers = qdata['answer'].tolist()

<ipython-input-36-4167a44b0538>:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  qdata = pd.read_csv('/content/drive/MyDrive/queries.csv')


In [74]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download the required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function to preprocess text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())
    # removing stop words and stemming
    tokens = [stemmer.stem(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Preprocess the paragraphs and queries
preprocessed_paragraphs = [preprocess_text(paragraph) for paragraph in paragraphs]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Vectorizing the data

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a TF-IDF vectorizerz
vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed paragraphs
paragraph_vectors = vectorizer.fit_transform(preprocessed_paragraphs)

## implementing cosine similarity

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

def context_gen(query):
  # x should be preprocessed query
  preprocessed_query = preprocess_text(query)
  # Preprocess and vectorize the query
  query_vector = vectorizer.transform([preprocessed_query])

  # Compute cosine similarity between the query and paragraphs
  similarities = cosine_similarity(query_vector, paragraph_vectors).flatten()

  # Sort the similarities and get the indices of the top 5 paragraphs
  top_indices = similarities.argsort()[-5:][::-1]

  ans = ""

  #concatanating all the strings into ans
  for idx in top_indices:
    ans = ans+ paragraphs[idx]

  return ans

context = context_gen(queries[0])
print(context)

The trophy comes in three parts - the cup itself, plus a lid and a base. There have been two designs of trophy in use, but five physical trophies have been presented. The original trophy, known as the "little tin idol", was 18 inches high and made by Martin, Hall & Co. It was stolen in 1895 and never recovered, and so was replaced by an exact replica, used until 1910. The FA decided to change the design after the 1909 winners, Manchester United, made their own replica, leading the FA to realise they did not own the copyright. This new, larger design was by Messers Fattorini and Sons, and was used from 1911. In order to preserve this original, from 1992 it was replaced by an exact replica, although this had to be replaced after just over two decades, after showing wear and tear from being handled more than in previous eras. This third replica, first used in 2014, was built heavier to withstand the increased handling. Of the four surviving trophies, only the 1895 replica has entered priv

## BERT model to process queries using context

using question answering pipeline of hugging face

In [60]:
from transformers import pipeline

qa_model = pipeline("question-answering")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [67]:
question = queries[0]
qa_model(question = question, context = context)['answer']

'the cup itself, plus a lid and a base'

In [63]:
answers[0]

"['- the cup itself, plus a lid and a base']"

In [68]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.4 MB/s

In [72]:
queries[0]

'What is the  parts of the trophy?'

In [70]:
import gradio as gr

def qa(query):
  context = context_gen(query)
  ans = qa_model(question = query, context = context)['answer']
  return ans
demo = gr.Interface(fn=qa, inputs="text", outputs="text")

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9ee19735fad92c9dfa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
